In [2]:
import cv2
import mediapipe as mp
import numpy as np

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [4]:
hands = mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    max_num_hands = 1
) 

In [5]:
def eulidean_distance(landmarkA, landmarkB):
    A = np.array([landmarkA.x, landmarkA.y])
    B = np.array([landmarkB.x, landmarkB.y])
    distance = np.linalg.norm(A-B)
    return distance

In [6]:
distance = np.linalg.norm(np.array([4, 3])-np.array([6, 2]))
distance

2.23606797749979

In [7]:
def add_distance(landmarks):
    data = []
    data.append(eulidean_distance(landmarks[4], landmarks[0]))
    data.append(eulidean_distance(landmarks[8], landmarks[0]))
    data.append(eulidean_distance(landmarks[12], landmarks[0]))
    data.append(eulidean_distance(landmarks[16], landmarks[0]))
    data.append(eulidean_distance(landmarks[20], landmarks[0]))
    data.append(eulidean_distance(landmarks[4], landmarks[8]))
    data.append(eulidean_distance(landmarks[4], landmarks[12]))
    data.append(eulidean_distance(landmarks[8], landmarks[12]))
    data.append(eulidean_distance(landmarks[12], landmarks[16]))
    data.append(eulidean_distance(landmarks[20], landmarks[16]))
    data.append(eulidean_distance(landmarks[8], landmarks[16]))
    data.append(eulidean_distance(landmarks[8], landmarks[20]))
    data.append(eulidean_distance(landmarks[12], landmarks[20]))
    data.append(eulidean_distance(landmarks[4], landmarks[12]))
    data.append(eulidean_distance(landmarks[4], landmarks[16]))
    data.append(eulidean_distance(landmarks[4], landmarks[20]))
    return data

In [8]:
def process(image, live=True):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    data = []
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0].landmark 
        data = add_distance(landmarks)
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        # Flip the image horizontally for a selfie-view display.
    if live:
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    return data

In [11]:
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Cannot read video stream")
        break
    data = np.array(process(frame)) 
    np.save(f"./training_data/space/data{'0'*(3-len(str(count)))+str(count)}.npy", data) 
    count+=1
    if count == 900:
        break
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()